In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import copy


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from keras import layers, Model, Sequential

In [4]:
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=False)

In [ ]:
images_count = 0
for images, labels in train_loader:
    plt.figure(figsize=(12,4))
    for i in range(16):
        plt.subplot(2,8,i+1)
        plt.imshow(images[i].numpy().squeeze(), cmap='gray')
        plt.axis('off')
        plt.title(f'Label: {labels[i].item()}')
    plt.show()
    images_count += len(images)
    if images_count >= 10:
        break


Create the autoencoder class

In [6]:
class AutoEncoder(Model):
    def __init__(self):
        super().__init__()
        self.encoder = Sequential([
            layers.Dense(128, activation='relu', input_shape=(784,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(32, activation='relu') #latent space
        ])

        self.decoder = Sequential([
            layers.Dense(32, activation='relu', input_shape=(32,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(784, activation='sigmoid')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        latent = self.latent(encoded)
        decoded = self.decoder(latent)
        return decoded

In [ ]:
model1 = AutoEncoder()
model1.summary()

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3, weight_decay=1e-5)

In [ ]:
epochs = 2000
outputs_list = []
best_model_loss = float('inf')
best_model_weights = None

patience = 7
patience_counter = 0

loss_list = []
accuracy_list = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}: ", end="")
    epoch_loss = 0

    for images, _ in train_loader:
        images = images.view(-1, 28*28).to(device)

        optimizer.zero_grad()
        outputs = model1(images, training=True)
        loss = loss_fn(outputs, images)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    loss_list.append(epoch_loss)

    print(f"Loss: {epoch_loss:.4f}")

    epoch_val_loss = 0
    epoch_val_accuracy = 0
    
    with torch.no_grad():
        for val_images, _ in test_loader:
            val_images = val_images.view(-1, 28*28).to(device)
            val_outputs = model1(val_images, training=False)
            val_loss = loss_fn(val_outputs, val_images)
            epoch_val_loss += val_loss.item()

            val_accuracy = F.mse_loss(val_outputs, val_images)
            epoch_val_accuracy += val_accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    accuracy_list.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model1.state_dict())
        patience_counter = 0
    
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation MSE: {avg_val_accuracy:.4f}")

In [ ]:
class AutoEncoder(Model):
    def __init__(self):
        super().__init__()
        self.encoder = Sequential([
            layers.Dense(128, activation='relu', input_shape=(784,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(32, activation='relu') #latent space
        ])

        self.decoder = Sequential([
            layers.Dense(32, activation='relu', input_shape=(32,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(784, activation='sigmoid')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        latent = self.latent(encoded)
        decoded = self.decoder(latent)
        return decoded

In [ ]:
model2 = AutoEncoder()
model2.summary()

In [ ]:
epochs = 2000
outputs_list = []
best_model_loss = float('inf')
best_model_weights = None

patience = 7
patience_counter = 0

loss_list = []
accuracy_list = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}: ", end="")
    epoch_loss = 0

    for images, _ in train_loader:
        images = images.view(-1, 28*28).to(device)

        optimizer.zero_grad()
        outputs = model2(images, training=True)
        loss = loss_fn(outputs, images)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    loss_list.append(epoch_loss)

    print(f"Loss: {epoch_loss:.4f}")

    epoch_val_loss = 0
    epoch_val_accuracy = 0
    
    with torch.no_grad():
        for val_images, _ in test_loader:
            val_images = val_images.view(-1, 28*28).to(device)
            val_outputs = model2(val_images, training=False)
            val_loss = loss_fn(val_outputs, val_images)
            epoch_val_loss += val_loss.item()

            val_accuracy = F.mse_loss(val_outputs, val_images)
            epoch_val_accuracy += val_accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    accuracy_list.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model2.state_dict())
        patience_counter = 0
    
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation MSE: {avg_val_accuracy:.4f}")

In [ ]:
class AutoEncoder(Model):
    def __init__(self):
        super().__init__()
        self.encoder = Sequential([
            layers.Dense(128, activation='relu', input_shape=(784,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(32, activation='relu') #latent space
        ])

        self.decoder = Sequential([
            layers.Dense(32, activation='relu', input_shape=(32,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(784, activation='sigmoid')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        latent = self.latent(encoded)
        decoded = self.decoder(latent)
        return decoded

In [ ]:
model3 = AutoEncoder()
model3.summary()

In [ ]:
epochs = 2000
outputs_list = []
best_model_loss = float('inf')
best_model_weights = None

patience = 7
patience_counter = 0

loss_list = []
accuracy_list = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}: ", end="")
    epoch_loss = 0

    for images, _ in train_loader:
        images = images.view(-1, 28*28).to(device)

        optimizer.zero_grad()
        outputs = model3(images, training=True)
        loss = loss_fn(outputs, images)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    loss_list.append(epoch_loss)

    print(f"Loss: {epoch_loss:.4f}")

    epoch_val_loss = 0
    epoch_val_accuracy = 0
    
    with torch.no_grad():
        for val_images, _ in test_loader:
            val_images = val_images.view(-1, 28*28).to(device)
            val_outputs = model3(val_images, training=False)
            val_loss = loss_fn(val_outputs, val_images)
            epoch_val_loss += val_loss.item()

            val_accuracy = F.mse_loss(val_outputs, val_images)
            epoch_val_accuracy += val_accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    accuracy_list.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model3.state_dict())
        patience_counter = 0
    
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation MSE: {avg_val_accuracy:.4f}")

In [ ]:
class AutoEncoder(Model):
    def __init__(self):
        super().__init__()
        self.encoder = Sequential([
            layers.Dense(128, activation='relu', input_shape=(784,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(32, activation='relu') #latent space
        ])

        self.decoder = Sequential([
            layers.Dense(32, activation='relu', input_shape=(32,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(784, activation='sigmoid')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        latent = self.latent(encoded)
        decoded = self.decoder(latent)
        return decoded

In [ ]:
model4 = AutoEncoder()
model4.summary()

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model4.parameters(), lr=1e-3, weight_decay=1e-5)

In [ ]:
epochs = 2000
outputs_list = []
best_model_loss = float('inf')
best_model_weights = None

patience = 7
patience_counter = 0

loss_list = []
accuracy_list = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}: ", end="")
    epoch_loss = 0

    for images, _ in train_loader:
        images = images.view(-1, 28*28).to(device)

        optimizer.zero_grad()
        outputs = model4(images, training=True)
        loss = loss_fn(outputs, images)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    loss_list.append(epoch_loss)

    print(f"Loss: {epoch_loss:.4f}")

    epoch_val_loss = 0
    epoch_val_accuracy = 0
    
    with torch.no_grad():
        for val_images, _ in test_loader:
            val_images = val_images.view(-1, 28*28).to(device)
            val_outputs = model4(val_images, training=False)
            val_loss = loss_fn(val_outputs, val_images)
            epoch_val_loss += val_loss.item()

            val_accuracy = F.mse_loss(val_outputs, val_images)
            epoch_val_accuracy += val_accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    accuracy_list.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model4.state_dict())
        patience_counter = 0
    
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation MSE: {avg_val_accuracy:.4f}")